In [3]:
import pandas as pd
import os
from datetime import datetime

In [5]:
def encontrar_dfs_con_valores_cero(dfs_by_country):
    """
    Encuentra los df con mas del 50% de valores cero.

    Parámetros:
        dfs_by_country (dic): dccionario con dos dfs por pais, uno masculino y otro femenino.
        

    Retorna: key_to_delete (lista): lista de claves de pais con aquellos que cumplen la condicion
   """
       
    # Lista para almacenar las claves de los DataFrames que se eliminarán
    keys_to_delete = []
    
    # Crear una copia de las claves del diccionario
    keys_copy = list(dfs_by_country.keys())
    
    # Iterar sobre las claves del diccionario
    for country in keys_copy:
        dfs_pais = dfs_by_country[country]
        for i, df in enumerate(dfs_pais):
            # Calcular el porcentaje de valores cero en 'total_obs_value'
            porcentaje_ceros = (df['total_obs_value'] == 0).mean() * 100
            
            # Verificar si el porcentaje de ceros es mayor al 50%
            if porcentaje_ceros > 50:
                # Agregar la clave del DataFrame a la lista de claves a eliminar
                keys_to_delete.append(country)
                break
    
        
    return keys_to_delete

In [6]:
def completar_series_trimestrales(ruta_carpeta):
    """
    Completa series temporales trimestrales faltantes en archivos CSV ubicados en un directorio
    de entrada y guarda las series completas en otro directorio.

    Parámetros:
        ruta_carpeta (str): La ruta al directorio que contiene los archivos CSV de series temporales.
        
    Retorna: None
   """
    # Verificar si la carpeta de salida existe, si no, crearla
    if not os.path.exists("../data/series"):
        os.makedirs("../data/series")
    
    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(ruta_carpeta)
    
    # Iterar sobre los archivos en la carpeta
    for archivo in archivos:
        # Verificar que el archivo es un archivo CSV
        if archivo.endswith('.csv'):
            # Construir la ruta completa al archivo
            ruta_archivo = os.path.join(ruta_carpeta, archivo)
            # Leer el archivo CSV
            df = pd.read_csv(ruta_archivo)
            # Convertir la columna 'year_month' a tipo datetime
            df['year_month'] = pd.to_datetime(df['year_month'])
            # Eliminar filas duplicadas si existen
            df = df.drop_duplicates(subset=['year_month'])
            # Obtener la menor y la mayor fecha presente en el DataFrame
            min_fecha = df['year_month'].min()
            max_fecha = df['year_month'].max()
            # Crear una serie temporal mensual entre la menor y la mayor fecha
            fechas_mensuales = pd.date_range(start=min_fecha, end=max_fecha, freq='Q')
            # Crear un DataFrame vacío con las fechas mensuales como índice
            df_mensual = pd.DataFrame(index=fechas_mensuales)
            # Rellenar la serie temporal mensual con los valores del trimestre correspondiente
            df_mensual['total'] = df.set_index('year_month').resample('Q').ffill()['total']
            # Guardar el DataFrame reducido como un archivo CSV
            ruta_nueva = os.path.join("../data/series", archivo)
            df_mensual.to_csv(ruta_nueva)
